In [20]:
import gdown

url = "https://drive.google.com/file/d/1wY2FSROsMASbX69kYBzn5_Y1DlcsJZpv/view?usp=drive_link"

file_id = url.split("/")[-2]
file_id

prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "Chest-CT-Scan-data.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1wY2FSROsMASbX69kYBzn5_Y1DlcsJZpv
From (redirected): https://drive.google.com/uc?/export=download&id=1wY2FSROsMASbX69kYBzn5_Y1DlcsJZpv&confirm=t&uuid=5e9b8cca-fd47-4dca-a78b-7d665f50be35
To: e:\datascience\Chest-CT-Scan-data.zip
100%|██████████| 1.63G/1.63G [07:19<00:00, 3.70MB/s]


'Chest-CT-Scan-data.zip'

In [54]:
import zipfile

zip_ref = zipfile.ZipFile("Chest-CT-Scan-data.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [55]:
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 1 directories and 0 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'.
There are 4 directories and 0 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'.
There are 0 directories and 3709 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Cyst'.
There are 0 directories and 5077 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Normal'.
There are 0 directories and 1377 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Stone'.
There are 0 directories and 2283 images in 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Tumor'.


In [57]:
import os
import shutil
from sklearn.model_selection import train_test_split

def create_train_test_folders(root_dir, classes, test_size=0.2, random_state=42):
    # Create train and test directories
    train_dir = os.path.join(root_dir, "train")
    test_dir = os.path.join(root_dir, "test")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for class_name in classes:
        # Get the list of images for the current class
        class_images = os.listdir(os.path.join(root_dir, root_dir, class_name))  # Adjusted path

        # Split the images into train and test sets
        train_images, test_images = train_test_split(class_images, test_size=test_size, random_state=random_state)

        # Create directories for the current class in train and test folders
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        # Copy images to the corresponding train and test class directories
        for image in train_images:
            source_path = os.path.join(root_dir, root_dir, class_name, image)  # Adjusted path
            destination_path = os.path.join(train_class_dir, image)
            shutil.copyfile(source_path, destination_path)

        for image in test_images:
            source_path = os.path.join(root_dir, root_dir, class_name, image)  # Adjusted path
            destination_path = os.path.join(test_class_dir, image)
            shutil.copyfile(source_path, destination_path)

# Example usage:
root_directory = 'CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'
class_names = ['Cyst', 'Normal', 'Stone', 'Tumor']

create_train_test_folders(root_directory, class_names)


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/train/"
test_dir = "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)


In [2]:
print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Testing images:")
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")
     

Training images:
Found 9955 images belonging to 4 classes.
Testing images:
Found 2491 images belonging to 4 classes.


In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Define IMAGE_SHAPE based on the expected input shape of the model
IMAGE_SHAPE = (224, 224)

def create_model(model_url, num_classes=4):
    """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
    
    Args:
      model_url (str): A TensorFlow Hub feature extraction URL.
      num_classes (int): Number of output neurons in output layer,
        should be equal to the number of target classes, default 4.
    
    Returns:
      An uncompiled Keras Sequential model with model_url as a feature
      extractor layer and Dense output layer with num_classes outputs.
    """
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                             trainable=False,  # freeze the underlying patterns
                                             name='feature_extraction_layer',
                                             input_shape=IMAGE_SHAPE + (3,))  # define the input image shape

    # Create our own model
    model = tf.keras.Sequential([
        feature_extractor_layer,  # use the feature extraction layer as the base
        layers.Dense(num_classes, activation='softmax', name='output_layer')  # create our own output layer
    ])

    return model

# Example usage
model1 = create_model("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-classification/versions/4", num_classes=4)


In [7]:
model1.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [8]:
resnet_history = model1.fit(train_data_10_percent,
                                  epochs=5,
                                  steps_per_epoch=len(train_data_10_percent),
                                  validation_data=test_data,
                                  validation_steps=len(test_data))

Epoch 1/5



312/312 [==============================] - 309s 978ms/step - loss: 0.5496 - accuracy: 0.8019 - val_loss: 0.3383 - val_accuracy: 0.8892
Epoch 2/5
312/312 [==============================] - 222s 710ms/step - loss: 0.2580 - accuracy: 0.9168 - val_loss: 0.2302 - val_accuracy: 0.9293
Epoch 3/5
312/312 [==============================] - 281s 903ms/step - loss: 0.1958 - accuracy: 0.9413 - val_loss: 0.1916 - val_accuracy: 0.9434
Epoch 4/5
312/312 [==============================] - 305s 976ms/step - loss: 0.1551 - accuracy: 0.9551 - val_loss: 0.1702 - val_accuracy: 0.9474
Epoch 5/5
312/312 [==============================] - 267s 855ms/step - loss: 0.1342 - accuracy: 0.9592 - val_loss: 0.1478 - val_accuracy: 0.9558


In [9]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 1001)              3540265   
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 4)                 4008      
                                                                 
Total params: 3544273 (13.52 MB)
Trainable params: 4008 (15.66 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________


In [10]:
hdf5_model_save_path = '/DATASCIENCE/model2.h5'

# Save the model in HDF5 format
model1.save(hdf5_model_save_path)

c:\Users\HP\anaconda3\envs\MLops\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
import mlflow
from mlflow.tensorflow import MLflowCallback

# Set the experiment name
experiment_name = "MLops_Mission_Impossible"

# Set the experiment
mlflow.set_experiment(experiment_name)


2024/01/06 20:13:29 INFO mlflow.tracking.fluent: Experiment with name 'MLops_Mission_Impossible' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/543761410144953044', creation_time=1704552209305, experiment_id='543761410144953044', last_update_time=1704552209305, lifecycle_stage='active', name='MLops_Mission_Impossible', tags={}>

In [24]:
# Save the trained model with the name "model3"
mlflow.tensorflow.save_model(model1, "model3")


2024/01/06 21:03:13 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: e:\datascience\model3\data\model\assets


INFO:tensorflow:Assets written to: e:\datascience\model3\data\model\assets
c:\Users\HP\anaconda3\envs\MLops\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [19]:
import mlflow
from mlflow.tensorflow import MLflowCallback

# Start the MLflow run
with mlflow.start_run() as run:

    # Log the model parameters
    mlflow.log_param("epochs", 5)
    mlflow.log_param("steps_per_epoch", len(train_data_10_percent))
    # Log other hyperparameters as needed

    # Train your model with MLflowCallback
    resnet_history = model1.fit(
        train_data_10_percent,
        epochs=5,
        steps_per_epoch=len(train_data_10_percent),
        validation_data=test_data,
        validation_steps=len(test_data),
        callbacks=[MLflowCallback(run)],
    )

    # Log the final metrics after training
    mlflow.log_metrics({
        "final_loss": resnet_history.history["loss"][-1],
        "final_accuracy": resnet_history.history["accuracy"][-1],
        "final_val_loss": resnet_history.history["val_loss"][-1],
        "final_val_accuracy": resnet_history.history["val_accuracy"][-1],
    })


Epoch 1/5
312/312 [==============================] - 224s 718ms/step - loss: 0.0730 - accuracy: 0.9798 - val_loss: 0.0984 - val_accuracy: 0.9655
Epoch 2/5
312/312 [==============================] - 225s 722ms/step - loss: 0.0660 - accuracy: 0.9832 - val_loss: 0.1013 - val_accuracy: 0.9655
Epoch 3/5
312/312 [==============================] - 222s 711ms/step - loss: 0.0617 - accuracy: 0.9826 - val_loss: 0.0824 - val_accuracy: 0.9771
Epoch 4/5
312/312 [==============================] - 227s 728ms/step - loss: 0.0555 - accuracy: 0.9856 - val_loss: 0.0852 - val_accuracy: 0.9763
Epoch 5/5
312/312 [==============================] - 330s 1s/step - loss: 0.0512 - accuracy: 0.9872 - val_loss: 0.0819 - val_accuracy: 0.9711


In [18]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow_hub as hub

# Function to load the model with custom object scope
def load_model_with_custom_objects(model_path):
    # Define a custom_objects dictionary to handle the KerasLayer
    custom_objects = {'KerasLayer': hub.KerasLayer}
    # Load the model with the custom_objects scope
    model = load_model(model_path, custom_objects=custom_objects)
    return model

# Load the trained model with custom object scope
model_path = 'model2.h5'
model = load_model_with_custom_objects(model_path)

# Define IMAGE_SHAPE based on the expected input shape of the model
IMAGE_SHAPE = (224, 224)

# Define class labels
class_labels = ['Cyst', 'Normal', 'Stone', 'Tumor']

# Function to preprocess an image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=IMAGE_SHAPE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  
    return img_array

def predict_class(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_label = class_labels[predicted_class_index]
    return predicted_class_label

# Example image path
image_path = 'Cyst- (35).jpg'

# Make a prediction
predicted_label = predict_class(image_path)

# Print the predicted label
print("Predicted label:", predicted_label)


1/1 [==============================] - 1s 602ms/step
Predicted label: Cyst
